# tootltips

In [6]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":92242,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    75  100    75    0     0    735      0 --:--:-- --:--:-- --:--:--   735


# Imports

In [1]:
from elasticsearch import Elasticsearch

# Client Elasticsearch

In [2]:
es_client = Elasticsearch()

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [3]:
index_name = "dpe_pre_1948_td005"
index_configurations = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0,
            "max_result_window": 500000,

        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "text_to_analyze": {
                "type": "text",
                "analyzer": "french"      
            },
            "category_index":{
                'type':'keyword'
            }
        }
    }
}

In [4]:
es_client.indices.delete(
    index=index_name,
)

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x00000141B1D10358>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x00000141B1D10358>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée)

In [ ]:
es_client.indices.create(
    index=index_name,
    body=index_configurations
)

# load tables

In [ ]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        print(dept_dir)
        annexe_dept_dir = annexe_dir / dept_dir.name
        annexe_dept_dir.mkdir(exist_ok=True, parents=True)
        # LOAD TABLES
        td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
        td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
        td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
        td006 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)

        break
        # ENVELOPPE PROCESSING


In [ ]:
mixte = ('mixte', 'combine', 'chauffage ecs','lie ','combine ',"idem")


gen_ecs_normalized_lib_matching_dict_ft = {
    "ECS thermodynamique electrique(PAC ou ballon)": [
        ('pompe chaleur', 'pac', 'thermodynamique', 'air extrait', 'air exterieur', 'air ambiant','thermo')],
    
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), ('electricite', 'electrique','elec.')],
    "ecs electrique indeterminee": [('electricite', 'electrique','elec.')],
    "ecs instantanee electrique": ['instantanee', ('electricite', 'electrique','elec.')],
    

}

for type_chaudiere, type_chaudiere_keys in zip(['standard', 'basse temperature', 'condensation', 'non déterminee'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs'), None]):
    for energie in ['fioul', 'gaz','autre(gpl/butane/propane)']:
        energie_keywords = energie
        if energie == 'autre(gpl/butane/propane)':
            energie_keywords = ('gpl', 'butane', 'propane')
        if type_chaudiere_keys is not None:
            gen_ecs_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords,
                                                                                            type_chaudiere_keys]
        else:
            gen_ecs_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords
                                                                                            ]


In [ ]:
gen_ecs_normalized_lib_matching_dict_ft.update({    'chaudiere non déterminee': ["chaudiere"],
    
    'production mixte gaz': ["gaz", mixte],
    'production mixte fioul': ["fioul", mixte],
    
    'production mixte indeterminee': [ mixte],
 'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"),
                                      "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],
})

In [ ]:
from generate_dpe_annexes_scripts.utils import strip_accents

In [ ]:
td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))


In [ ]:
def clean_valeur_renseignee(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t",  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [ ]:
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_valeur_renseignee(x))

# feed es

In [ ]:
from elasticsearch.helpers import bulk,parallel_bulk

In [ ]:
# TODO : optimize

In [ ]:
def gendata():
    for id_,vr in zip(td005_ecs.id,td005_ecs.valeur_renseignee):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

bulk(es_client, gendata())

# catégoriser mes documents

In [ ]:
s_list = list()
for label,char_list in  gen_ecs_normalized_lib_matching_dict_ft.items():
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        if isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
    search_instruction = ' AND '.join(new_char_list)
    search_body = {
        "query": {
            "query_string": {
                "query": search_instruction,
                "default_field": "text_to_analyze"
            },

        },

    }
    #es_client.count(index=index_name, body=search_body)

    a_dict=es_client.search(index=index_name, body=search_body,size=500000)

    hits = a_dict['hits']['hits']
    s=pd.Series(index=[el['_source']['id'] for el in hits])
    s[:]=label
    s_list.append(s)

In [ ]:
s_all=pd.concat(s_list)

In [ ]:
s_all

In [ ]:
df=s_all.to_frame('label')

In [ ]:
df.index.name = 'id'

In [ ]:
df = df.reset_index().drop_duplicates(keep='first',subset=['id'])

In [ ]:
m=td005_ecs.merge(df,on='id',how='left')

In [ ]:
m.label.value_counts()

In [136]:
for el in m.loc[m.label =='production mixte indeterminee'].valeur_renseignee.head(1000):
    print(el)
    print(any([s in el for s in pac]))

 - lie au chauffage
False
 - production ecs par generateur mixte  chauffage et ecs)
False
 
False
 - production ecs par chauffe-eau gaz
False
 - production ecs par generateur mixte  chauffage et ecs)  production assuree par la generation : generation n 1   generateur 1 ) installation individuelle production a accumulation  1 ballon de 50 litres  production dans le volume habitable les pieces desservies sont non contigues  
False
ecs n 1   type d installation : sans ecs solaire   energie : gaz   type equipement : chauffe eau gaz   contiguite des pieces : pieces non contigues   position de la production : en volume chauffe   isolation du reseau : reseau non isole   volume de stockage : 200   veilleuse : non   alimentation : heure pleine   
False
 - production ecs par generateur mixte  chauffage et ecs)
False
 
False
aucune installation collective idem chauffage
False
 - production ecs par generateur mixte  chauffage et ecs)
False
idem chauffage aucun equipement individuel
False
idem chau